In [ ]:
import numpy as np
from numpy.linalg import norm
from readlammps import ReadLammpsData
from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation as R

In [ ]:
def generate_random_location(Lx, Ly, Lz, recenter=True):
    """
    generate a random location within a given box
    """  
    if recenter == True:
        x = np.random.rand()*Lx-Lx/2
        y = np.random.rand()*Ly-Ly/2
        z = np.random.rand()*Lz-Lz/2
    else:
        x = np.random.rand()*Lx
        y = np.random.rand()*Ly
        z = np.random.rand()*Lz   
    return x, y, z

In [ ]:
def generate_random_orientation(XYZ_initial):
    """
    generate 3D aleatory rotation for molecule coordinate
    """
    rotation_degrees = np.random.rand()*90
    rotation_radians = np.radians(rotation_degrees)
    rotation_axis = np.array([np.random.rand(), 
                              np.random.rand(), 
                              np.random.rand()])
    rotation_axis /= np.linalg.norm(rotation_axis)
    rotation_vector = rotation_radians * rotation_axis
    rotation = R.from_rotvec(rotation_vector)
    XYZ_rotated = rotation.apply(XYZ_initial)
    return XYZ_rotated

In [ ]:
def search_closest_neighbor(XYZ_neighbor, XYZ_molecule, Lx, Ly, Lz):
    """
        Search neighbor in a box and return the closest distance with a molecule
        
        Periodic boundary conditions are automatically accounted
    """
    box = np.array([Lx, Ly, Lz])
    min_distance = np.max(box)
    for XYZ_atom in XYZ_molecule:
        dxdydz = np.remainder(XYZ_neighbor - XYZ_atom + box/2., box) - box/2.
        min_distance = np.min([min_distance,np.min(norm(dxdydz,axis=1))])
    return min_distance

In [ ]:
# import PEG data
peg_data = ReadLammpsData("peg/peg.data", verbose = False)
Lx, Ly, Lz = peg_data.box_dimensions.T[1] - peg_data.box_dimensions.T[0]
# import water data
water_data = ReadLammpsData("water/h2o.data", verbose = False)
# import ethanol data
ethanol_data = ReadLammpsData("ethanol/ethanol.data", verbose = False)


#### Insert PEG

In [ ]:
atom_data = []
atom_coordinate = []
cpt_atom = 0
cpt_molecule = 0
cpt_peg = 0
type_peg = 0
atom_types = []
molecule_ids = []
molecule_names = []
# add atoms
for ids, molid, types, charges, coords in zip(peg_data.atom_ids, peg_data.molecule_labels, peg_data.atom_labels, peg_data.atom_charges, peg_data.coords):
    cpt_atom += 1
    line = str(ids)+' '+str(molid)+' '+str(types)+' '+str(charges)+' '+str(coords[0])+' '+str(coords[1])+' '+str(coords[2])
    atom_data.append(line)
    atom_coordinate.append(coords)
    cpt_molecule = np.max([cpt_molecule, molid])
    cpt_peg = np.max([cpt_peg, ids])
    type_peg = np.max([type_peg, types])
    molecule_ids.append(molid)
    molecule_names.append("PEG")
    atom_types.append(types)
# add bond
cpt_bond = 0
bond_data = []
bond_type_peg = 0
for labels, bonds in zip(peg_data.bond_labels, peg_data.bonds):
    cpt_bond += 1
    line = str(cpt_bond)+' '+str(labels)+' '+str(bonds[0])+' '+str(bonds[1])
    bond_data.append(line)
    bond_type_peg = np.max([bond_type_peg, labels])
# add angle
cpt_angle = 0
angle_data = []
angle_type_peg = 0
for labels, angles in zip(peg_data.angle_labels, peg_data.angles):
    cpt_angle += 1
    line = str(cpt_angle)+' '+str(labels)+' '+str(angles[0])+' '+str(angles[1])+' '+str(angles[2])
    angle_data.append(line)
    angle_type_peg = np.max([angle_type_peg, labels])
dihedral_data = []
# add dihedral
dihedral_data = []
cpt_dihedral = 0
dihedral_type_peg = 0
for labels, dihedrals in zip(peg_data.dihedral_labels, peg_data.dihedrals):
    cpt_dihedral += 1
    line = str(cpt_dihedral)+' '+str(labels)+' '+str(dihedrals[0])+' '+str(dihedrals[1])+' '+str(dihedrals[2])+' '+str(dihedrals[3])
    dihedral_data.append(line)
    dihedral_type_peg = np.max([dihedral_type_peg, labels])

#### Insert ethanol

In [ ]:
desired_ethanol = 200
cut_off = 1.5
number_ethanol = 0
type_ethanol = 0
bond_type_ethanol = 0
angle_type_ethanol = 0
dihedral_type_ethanol = 0
while number_ethanol < desired_ethanol:
    x, y, z = generate_random_location(Lx, Ly, Lz)
    rotated_ethanol_coordinate = generate_random_orientation(ethanol_data.coords)
    ethanol_coordinate = rotated_ethanol_coordinate + np.array([x, y, z])
    closest_distance = search_closest_neighbor(np.array(atom_coordinate),
                                               ethanol_coordinate,
                                               Lx, Ly, Lz)
    
    if closest_distance<cut_off:
        overlap = True
    else:
        overlap = False

    if overlap is False:
        cpt_molecule += 1
        number_ethanol += 1
        cpt_atom_temp = cpt_atom
        for ids, types, charges, coords in zip(ethanol_data.atom_ids, ethanol_data.atom_labels, ethanol_data.atom_charges, ethanol_coordinate):
            cpt_atom += 1
            line = str(ids+cpt_atom_temp)+' '+str(cpt_molecule)+' '+str(types+type_peg)+' '+str(charges)+' '+str(coords[0])+' '+str(coords[1])+' '+str(coords[2])
            atom_data.append(line)
            atom_coordinate.append(coords)
            type_ethanol = np.max([type_ethanol, types])
            molecule_ids.append(cpt_molecule)
            molecule_names.append("eth")
            atom_types.append(types+type_peg)
    
        for labels, bonds in zip(ethanol_data.bond_labels, ethanol_data.bonds):
            cpt_bond += 1
            line = str(cpt_bond)+' '+str(labels+bond_type_peg)+' '+str(bonds[0]+cpt_atom_temp)+' '+str(bonds[1]+cpt_atom_temp)
            bond_data.append(line)
            bond_type_ethanol = np.max([bond_type_ethanol, labels])

        for labels, angles in zip(ethanol_data.angle_labels, ethanol_data.angles):
            cpt_angle += 1
            line = str(cpt_angle)+' '+str(labels+angle_type_peg)+' '+str(angles[0]+cpt_atom_temp)+' '+str(angles[1]+cpt_atom_temp)+' '+str(angles[2]+cpt_atom_temp)
            angle_data.append(line)
            angle_type_ethanol = np.max([angle_type_ethanol, labels])

        for labels, dihedrals in zip(ethanol_data.dihedral_labels, ethanol_data.dihedrals):
            cpt_dihedral += 1
            line = str(cpt_dihedral)+' '+str(labels+dihedral_type_peg)+' '+str(dihedrals[0]+cpt_atom_temp)+' '+str(dihedrals[1]+cpt_atom_temp)+' '+str(dihedrals[2]+cpt_atom_temp)+' '+str(dihedrals[3]+cpt_atom_temp)
            dihedral_data.append(line)
            dihedral_type_ethanol = np.max([dihedral_type_ethanol, labels])

#### Insert water

In [ ]:
desired_water = 500
cut_off = 1.5
number_water = 0
type_water = 0
bond_type_water = 0
angle_type_water = 0
dihedral_type_water = 0
while number_water < desired_water:
    x, y, z = generate_random_location(Lx, Ly, Lz)
    rotated_water_coordinate = generate_random_orientation(water_data.coords)
    water_coordinate = rotated_water_coordinate + np.array([x, y, z])
    closest_distance = search_closest_neighbor(np.array(atom_coordinate),
                                               water_coordinate,
                                               Lx, Ly, Lz)
    
    if closest_distance<cut_off:
        overlap = True
    else:
        overlap = False

    if overlap is False:
        cpt_molecule += 1
        number_water += 1
        cpt_atom_temp = cpt_atom
        for ids, types, charges, coords in zip(water_data.atom_ids, water_data.atom_labels, water_data.atom_charges, water_coordinate):
            cpt_atom += 1
            line = str(ids+cpt_atom_temp)+' '+str(cpt_molecule)+' '+str(types+type_peg+type_ethanol)+' '+str(charges)+' '+str(coords[0])+' '+str(coords[1])+' '+str(coords[2])
            atom_data.append(line)
            atom_coordinate.append(coords)
            type_water = np.max([type_water, types])
            molecule_ids.append(cpt_molecule)
            molecule_names.append("wat")
            atom_types.append(types+type_peg+type_ethanol)
    
        for labels, bonds in zip(water_data.bond_labels, water_data.bonds):
            cpt_bond += 1
            line = str(cpt_bond)+' '+str(labels+bond_type_peg+bond_type_ethanol)+' '+str(bonds[0]+cpt_atom_temp)+' '+str(bonds[1]+cpt_atom_temp)
            bond_data.append(line)
            bond_type_water = np.max([bond_type_water, labels])

        for labels, angles in zip(water_data.angle_labels, water_data.angles):
            cpt_angle += 1
            line = str(cpt_angle)+' '+str(labels+angle_type_peg+angle_type_ethanol)+' '+str(angles[0]+cpt_atom_temp)+' '+str(angles[1]+cpt_atom_temp)+' '+str(angles[2]+cpt_atom_temp)
            angle_data.append(line)
            angle_type_water = np.max([angle_type_water, labels])

In [ ]:
# write LAMMPS data file
f = open("mixture.data", "w")
f.write('# LAMMPS data file \n\n')
f.write(str(cpt_atom)+' atoms\n')
f.write(str(cpt_bond)+' bonds\n')
f.write(str(cpt_angle)+' angles\n')
f.write(str(cpt_dihedral)+' dihedrals\n')
f.write('\n')
f.write(str(int(type_peg+type_ethanol+type_water))+' atom types\n')
f.write(str(int(bond_type_peg+bond_type_ethanol+bond_type_water))+' bond types\n')
f.write(str(int(angle_type_peg+angle_type_ethanol+angle_type_water))+' angle types\n')
f.write(str(int(dihedral_type_peg+dihedral_type_ethanol))+' dihedral types\n')
f.write('\n')
f.write(str(-Lx/2)+' '+str(Lx/2)+' xlo xhi\n')
f.write(str(-Ly/2)+' '+str(Ly/2)+' ylo yhi\n')
f.write(str(-Lz/2)+' '+str(Lz/2)+' zlo zhi\n')
f.write('\n')
f.write('Atoms\n')
f.write('\n')
for line in atom_data:
    f.write(line+'\n')
f.write('\n')
f.write('Bonds\n')
f.write('\n')
for line in bond_data:
    f.write(line+'\n')
f.write('\n')
f.write('Angles\n')
f.write('\n')
for line in angle_data:
    f.write(line+'\n')
f.write('\n')
f.write('Dihedrals\n')
f.write('\n')
for line in dihedral_data:
    f.write(line+'\n')
f.close()

In [ ]:
types = ["C", "O", "H", "O", "H", "C", "H", "H", "C", "O", "H", "O"]

In [ ]:
# write GROMACS gro file
f = open('mixture.gro', 'w')
f.write('PEG-ethanol-water SYSTEM\n')
f.write(str(cpt_atom)+'\n')
n = 0
for residue_id, residue_name, atom_type, coordinate in zip(molecule_ids, molecule_names, atom_types, atom_coordinate):
    n += 1
    f.write("{: >5}".format(str(residue_id))) # residue number (5 positions, integer) 
    f.write("{: >5}".format(residue_name)) # residue name (5 characters)
    f.write("{: >5}".format(types[atom_type-1])) # atom name (5 characters) 
    f.write("{: >5}".format(str(np.int32(n)))) # atom number (5 positions, integer)

    f.write("{: >8}".format(str("{:.3f}".format(coordinate[0])))) # position (in nm, x y z in 3 columns, each 8 positions with 3 decimal places)
    f.write("{: >8}".format(str("{:.3f}".format(coordinate[1])))) # position (in nm, x y z in 3 columns, each 8 positions with 3 decimal places) 
    f.write("{: >8}".format(str("{:.3f}".format(coordinate[2])))) # position (in nm, x y z in 3 columns, each 8 positions with 3 decimal places) 
    f.write("\n")
f.write("{: >10}".format(str("{:.5f}".format(Lx/10))))
f.write("{: >10}".format(str("{:.5f}".format(Ly/10))))
f.write("{: >10}".format(str("{:.5f}".format(Lz/10))))
f.write("\n")
f.close()